In [1]:
from __future__ import absolute_import
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [20]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [21]:
class word2vec_basic:
    '''
    
    This version has not removed stop words.
    Args:
    .
    word embeddings: self.final_embeddings
    words dictionary: self.reversed_dictionary, {0: word_1, 1: word_2 ...}
    '''
    
    def __init__(self, vocabulary_size=50000, read_filename='text8.zip', expected_bytes=31344016, batch_size=128, 
                 embedding_size=128, skip_window=1, num_skips=2):
        # download the dataset
        self.url='http://mattmahoney.net/dc/'
        self.vocabulary_size=vocabulary_size
        self.read_filename=read_filename
        self.expected_bytes=expected_bytes
        
        # construct dataset to be used
        self.filename=self.maybe_download()
        self.words=self.read_data()
        self.data, self.count, self.dictionary, self.reverse_dictionary=self.build_dataset()
        self.data_index=0

        # model parameters
        self.batch_size = batch_size
        self.embedding_size = embedding_size  # Dimension of the embedding vector.
        self.skip_window = skip_window       # How many words to consider left and right.
        self.num_skips = num_skips         # How many times to reuse an input to generate a label.

        # We pick a random validation set to sample nearest neighbors. Here we limit the
        # validation samples to the words that have a low numeric ID, which by
        # construction are also the most frequent.
        self.valid_size = 16     # Random set of words to evaluate similarity on.
        self.valid_window = 100  # Only pick dev samples in the head of the distribution.

        # replace=False means that we could not choose one for more than one time 
        self.valid_examples = np.random.choice(self.valid_window, self.valid_size, replace=False)
        self.num_sampled = 64    # Number of negative examples to sample.
        
        # build the graph
        self.build_graph()

    def maybe_download(self, filename=None, expected_bytes=None):
        if (filename is None) or (expected_bytes is None):
            filename=self.read_filename
            expected_bytes=self.expected_bytes
        if not os.path.exists(filename):
            filename, _=urllib.request.urlretrieve(url+filename, filename)
        stat_info=os.stat(filename)
        if not stat_info.st_size==expected_bytes:
            print('Wrong size:')
            print(stat_info.st_size)
            raise Exception('Failed to verify the dataset, could you just download from your browser?')
        else:
            print('Found and verified', filename)
        return filename

    # now read the data
    def read_data(self, filename=None):
        """

        Extract the first file enclosed in a zip file as a list of words
        """
        if filename is None:
            filename=self.filename
        with zipfile.ZipFile(filename) as f:
            data = f.read(f.namelist()[0]).split()
        return data

    # Step 2: build the dictionary and replace all those rare words with a UNK token
    def build_dataset(self, words=None):
        if words is None:
            words=self.words
        # count: a list of tuples (word_i, frequency_i)
        #    later we will replace count['UNK'] with its true frequency
        unk_count=0
        count=[['UNK', -1]]    # not to use a tuple because tuple is immutable 
        count.extend(collections.Counter(words).most_common(self.vocabulary_size-1))

        # dictionary: keys are the words in count, while values are the index of each words
        #     e.g. dictionary={'word_0': 0, 'word_1': 1, ...}
        dictionary=dict()
        for word, _ in count:
            dictionary[word]=len(dictionary)

        # data: a list with the same length of the input of this function -- 'words'
        #    each element data_i is this: dictionary[word_i in words] 
        #    -- index of each word in the dictionary
        data=list()
        for word in words:
            if word in dictionary:
                data.append(dictionary[word])
            else:
                data.append(0)
                unk_count+=1

        #
        count[0][1]=unk_count
        reverse_dictionary=dict(zip(dictionary.values(), dictionary.keys()))

        return data, count, dictionary, reverse_dictionary

    # Step 3: Function to generate a training batch for the skip-gram model.
    def generate_batch(self, batch_size=128, num_skips=2, skip_window=1):
        #global data_index
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window
        batch = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
        span = 2 * skip_window + 1 # [ skip_window target skip_window ]
        buffer = collections.deque(maxlen=span)
        for _ in range(span):
            buffer.append(self.data[self.data_index])
            self.data_index = (self.data_index + 1) % len(self.data)
        for i in range(batch_size // num_skips):
            target = skip_window  # target label at the center of the buffer
            targets_to_avoid = [ skip_window ]
            for j in range(num_skips):
                while target in targets_to_avoid:
                    target = random.randint(0, span - 1)
                targets_to_avoid.append(target)
                batch[i * num_skips + j] = buffer[skip_window]
                labels[i * num_skips + j, 0] = buffer[target]
            buffer.append(self.data[self.data_index])
            self.data_index = (self.data_index + 1) % len(self.data)
        return batch, labels
    
    # build the graph
    def build_graph(self):
        self.graph = tf.Graph()

        with self.graph.as_default():

            # Input data.
            self.train_inputs = tf.placeholder(tf.int32, shape=[self.batch_size])
            self.train_labels = tf.placeholder(tf.int32, shape=[self.batch_size, 1])
            self.valid_dataset = tf.constant(self.valid_examples, dtype=tf.int32)

            # Ops and variables pinned to the CPU because of missing GPU implementation
            with tf.device('/cpu:0'):
                # Look up embeddings for inputs.
                self.embeddings = tf.Variable(tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0))
                self.embed = tf.nn.embedding_lookup(self.embeddings, self.train_inputs)

            # Construct the variables for the NCE loss
            self.nce_weights = tf.Variable(tf.truncated_normal([self.vocabulary_size, self.embedding_size],
                                    stddev=1.0 / math.sqrt(self.embedding_size)))
            self.nce_biases = tf.Variable(tf.zeros([self.vocabulary_size]))

            # Compute the average NCE loss for the batch.
            # tf.nce_loss automatically draws a new sample of the negative labels each
            # time we evaluate the loss.
            self.loss = tf.reduce_mean(tf.nn.nce_loss(self.nce_weights, self.nce_biases, self.embed, self.train_labels,
                             self.num_sampled, self.vocabulary_size))

            # Construct the SGD optimizer using a learning rate of 1.0.
            self.optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(self.loss)

            # Compute the cosine similarity between minibatch examples and all embeddings.
            norm = tf.sqrt(tf.reduce_sum(tf.square(self.embeddings), 1, keep_dims=True))
            self.normalized_embeddings = self.embeddings / norm
            #valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
            #similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
            
    # train
    def train(self):
        num_steps=10001
        self.session = tf.InteractiveSession(graph=self.graph)
            
        tf.initialize_all_variables().run()
        print('Initialized')

        self.average_loss=0.0
        for step in range(num_steps):
            batch_inputs, batch_labels=self.generate_batch(self.batch_size, self.num_skips, self.skip_window)
            feed_dict={self.train_inputs: batch_inputs, self.train_labels: batch_labels}
            _, loss_val=self.session.run([self.optimizer, self.loss], feed_dict)
            self.average_loss+=loss_val

            if step%1000 == 0:
                print('average loss at step %s'%step, self.average_loss/step)
        self.average_loss/=num_steps
        self.final_embeddings=self.normalized_embeddings.eval()
            
    # visualize the word2vec
    def plot_with_labels(self, low_dim_embs, labels, filename='tsne.png'):
        assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
        plt.figure(figsize=(18, 18))  #in inches
        for i, label in enumerate(labels):
            x, y = low_dim_embs[i,:]
            plt.scatter(x, y)
            plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
        plt.savefig(filename)
        
    # visualization
    def visualize(self, plot_only=50):    
        tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
        low_dim_embs = tsne.fit_transform(self.final_embeddings[:plot_only,:])
        labels = [self.reverse_dictionary[i] for i in xrange(plot_only)]
        self.plot_with_labels(low_dim_embs, labels)
       
    # close the session
    def close(self):
        self.session.close()

In [22]:
m=word2vec_basic()

Found and verified text8.zip


In [23]:
m.train()

Initialized
average loss at step 0 inf
average loss at step 1000 145.297402325
average loss at step 2000 114.427349751
average loss at step 3000 96.1082074471
average loss at step 4000 83.5120459063
average loss at step 5000 73.8588944111
average loss at step 6000 66.7074071606
average loss at step 7000 60.6760791579
average loss at step 8000 55.8157460918
average loss at step 9000 51.7686697176
average loss at step 10000 48.2225678111


In [24]:
m.visualize()

/usr/lib/python2.7/dist-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [41]:
m.reverse_dictionary[2]

'of'